In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import json
import time

In [2]:
CHARS = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
INDEX_TO_CHAR = [w for w in CHARS]
DICT_LENGHT = len(INDEX_TO_CHAR)
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
SHIFT = 3

Класс шифровщик

In [4]:
class CaesarСipher:
    def __init__(self, dictionary=CHARS, key_shift=SHIFT):
        self.dictionary = dictionary
        self.key_shift = key_shift

    def encode(self, phrase: 'str фраза для кодирования') -> str:
        upper_ind = []

        for ind, char in enumerate(phrase):
            if char.istitle():
                upper_ind.append(ind)
        phrase = phrase.lower()

        encoded_phrase = ''
        for ind, char in enumerate(phrase):
            if char in self.dictionary:
                new_index = self.dictionary.find(char) + self.key_shift
                if new_index < len(CHARS) - 1:
                    encoded_phrase += self.dictionary[new_index]
                else:
                    encoded_phrase += self.dictionary[new_index - (len(CHARS))]
            else:
                encoded_phrase += char

        res_phrase = ''
        for ind, char in enumerate(encoded_phrase):
            if ind in upper_ind:
                res_phrase += char.upper()
            else:
                res_phrase += char

        return res_phrase

Класс обработчик текста. Подготавливает строку к индексированию, сохраняет индексы символов, которые не входят в алфавит, а так же сохраняет индексы заглавных букв.

In [6]:
class TextProcessor:
    def __init__(self, dictionary=CHARS):
        self.return_phrase = ''
        self.index_symbols = []
        self.dictionary = dictionary

    def text_processor_in(self, phrase: str) -> str | list | list:
        """
        Функция принимает на вход строку.
        На выход подаёт строку в нижнем регистре,
        символы, которые не входят в алфавит, включая пробелы
        подаются во вторую переменную списком,
        индексы заглавных букв передаются списком в 3ю переменную
        """
        upper_ind = []
        
        for ind, char in enumerate(phrase):
            if char.istitle():
                upper_ind.append(ind)
        phrase = phrase.lower()
        
        for ind, char in enumerate(phrase):
            if char in self.dictionary:
                self.return_phrase += char
            else:
                self.index_symbols.append([ind, char])
                
        return self.return_phrase, self.index_symbols, upper_ind

    def text_processor_out(self, phrase: str, index_symbols: list, up_ind: list) -> str:
        '''
        Функция принимает на вход строку, список с иными символами и
        список с индексами заглавных букв.
        Возвращает отформатированную строку
        '''
        phrase = list(phrase)
        for ind, char in index_symbols:
            phrase.insert(ind, char)

        res_phrase = ''
        for ind, char in enumerate(phrase):
            if ind in up_ind:
                res_phrase += char.upper()
            else:
                res_phrase += char
            
        return res_phrase

Блок для создания синтетического тренировочного датасета. В данном блоке я создаю большой датасет, в котором будут списки по 10 индексов и записываются как y_train, а так же при помощи функции caesar_cipher сдвигаем индексы и записываем в X_train и объединяем в train тензор

In [8]:
def text_to_numbers(text):
    numbers = []
    for char in text:
        index = CHARS.find(char)
        numbers.append(index)
    return numbers

def numbers_to_text(numbers):
    text = ''
    for number in numbers:
        if 0 <= number < DICT_LENGHT:  # Проверяем, что индекс находится в пределах алфавита
            text += CHARS[number]
    return text

def caesar_cipher(text, shift):
    numbers = text_to_numbers(text)
    shifted_numbers = [(x - shift) % len(CHARS) for x in numbers]
    return numbers_to_text(shifted_numbers)

X_train = []
y_train = []
for i in range(1000):  # Увеличиваем количество обучающих примеров
    text = ''.join(np.random.choice(list(CHARS), size=10))  # Разная длина текста
    encrypted_text = caesar_cipher(text, SHIFT)
    X_train.append(text_to_numbers(text))
    y_train.append(text_to_numbers(encrypted_text))

X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
train = torch.stack([X_train, y_train], dim=1)

In [9]:
class DecrypterModel(torch.nn.Module):
    def __init__(self):
        super(DecrypterModel, self).__init__()
        self.embed = torch.nn.Embedding(DICT_LENGHT, 30)
        self.rnn = torch.nn.RNN(30, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, DICT_LENGHT)
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, a = self.rnn(embed)
        out = self.linear(o)
        return out

model = DecrypterModel()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)


In [10]:
for ep in range(15):
    start = time.time()
    train_loss = 0.
    train_passed = 0.

    for line in train:
        X = line[0]
        y = line[1]

        optimizer.zero_grad()
        answers = model.forward(X)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))


Epoch 0. Time: 1.132, Train loss: 0.357
Epoch 1. Time: 1.717, Train loss: 0.014
Epoch 2. Time: 1.877, Train loss: 0.007
Epoch 3. Time: 1.746, Train loss: 0.005
Epoch 4. Time: 1.850, Train loss: 0.004
Epoch 5. Time: 1.813, Train loss: 0.003
Epoch 6. Time: 1.869, Train loss: 0.002
Epoch 7. Time: 1.909, Train loss: 0.002
Epoch 8. Time: 1.789, Train loss: 0.002
Epoch 9. Time: 1.842, Train loss: 0.002
Epoch 10. Time: 1.669, Train loss: 0.001
Epoch 11. Time: 1.550, Train loss: 0.001
Epoch 12. Time: 1.511, Train loss: 0.001
Epoch 13. Time: 1.797, Train loss: 0.001
Epoch 14. Time: 1.817, Train loss: 0.001


In [11]:
def encrypter(input_phrase: str) -> str:
    '''
    Принимает на вход зашиврованную фразу
    Выход - расшифрованная фраза
    '''
    tp = TextProcessor(dictionary=CHARS)
    phrase, symb, up_ind = tp.text_processor_in(input_phrase)
    X = []
    for char in phrase:
        X.append(CHAR_TO_INDEX[char])
    X = torch.tensor(X)
    prediction = model(X)
    result_tensor = torch.argmax(prediction, dim=1)
    result = ''
    for ind in result_tensor:
        result += INDEX_TO_CHAR[ind]

    return tp.text_processor_out(result, symb, up_ind)
    
    

In [12]:
poem ='''
У лукоморья дуб зелёный;
Златая цепь на дубе том:
И днём и ночью кот учёный
Всё ходит по цепи кругом;
Идёт направо — песнь заводит,
Налево — сказку говорит.
Там чудеса: там леший бродит,
Русалка на ветвях сидит;
Там на неведомых дорожках
Следы невиданных зверей;
Избушка там на курьих ножках
Стоит без окон, без дверей;
Там лес и дол видений полны;
Там о заре прихлынут волны
На брег песчаный и пустой,
И тридцать витязей прекрасных
Чредой из вод выходят ясных,
И с ними дядька их морской;
Там королевич мимоходом
Пленяет грозного царя;
Там в облаках перед народом
Через леса, через моря
Колдун несёт богатыря;
В темнице там царевна тужит,
А бурый волк ей верно служит;
Там ступа с Бабою Ягой
Идёт, бредёт сама собой,
Там царь Кащей над златом чахнет;
Там русский дух… там Русью пахнет!
И там я был, и мёд я пил;
У моря видел дуб зелёный;
Под ним сидел, и кот учёный
Свои мне сказки говорил.
'''

In [13]:
cc = CaesarСipher(dictionary=CHARS)
encoded_phrase = cc.encode(poem)
print(encoded_phrase)


Ц оцнспсуяв жцд кзоирюм;
Когхгв щзтя рг жцдз хсп:
Л жрип л рсъяб нсх цъирюм
Ефи шсжлх тс щзтл нуцёсп;
Лжих ргтугес — тзфря кгесжлх,
Ргозес — фнгкнц ёсесулх.
Хгп ъцжзфг: хгп озылм дусжлх,
Уцфгонг рг езхевш флжлх;
Хгп рг рзезжспюш жсусйнгш
Фозжю рзелжгррюш кезузм;
Лкдцынг хгп рг нцуялш рсйнгш
Фхслх дзк снср, дзк жезузм;
Хгп озф л жсо елжзрлм тсорю;
Хгп с кгуз тулшоюрцх есорю
Рг дузё тзфъгрюм л тцфхсм,
Л хулжщгхя елхвкзм тузнугфрюш
Ъузжсм лк есж еюшсжвх вфрюш,
Л ф рлпл жвжянг лш псуфнсм;
Хгп нсусозелъ плпсшсжсп
Тозрвзх ёускрсёс щгув;
Хгп е сдогнгш тзузж ргусжсп
Ъзузк озфг, ъзузк псув
Нсожцр рзфих дсёгхюув;
Е хзпрлщз хгп щгузерг хцйлх,
Г дцуюм есон зм езурс фоцйлх;
Хгп фхцтг ф Дгдсб Вёсм
Лжих, дузжих фгпг фсдсм,
Хгп щгуя Нгьзм ргж когхсп ъгшрзх;
Хгп уцффнлм жцш… хгп Уцфяб тгшрзх!
Л хгп в дюо, л пиж в тло;
Ц псув елжзо жцд кзоирюм;
Тсж рлп флжзо, л нсх цъирюм
Фесл прз фнгкнл ёсесуло.



In [14]:
result = encrypter(encoded_phrase)

In [15]:
print(result)


У лукоморья дуб зелёный;
Златая цепь на дубе том:
И днём и ночью кот учёный
Всё ходит по цепи кругом;
Идёт направо — песнь заводит,
Налево — сказку говорит.
Там чудеса: там леший бродит,
Русалка на ветвях сидит;
Там на неведомых дорожках
Следы невиданных зверей;
Избушка там на курьих ножках
Стоит без окон, без дверей;
Там лес и дол видений полны;
Там о заре прихлынут волны
На брег песчаный и пустой,
И тридцать витязей прекрасных
Чредой из вод выходят ясных,
И с ними дядька их морской;
Там королевич мимоходом
Пленяет грозного царя;
Там в облаках перед народом
Через леса, через моря
Колдун несёт богатыря;
В темнице там царевна тужит,
А бурый волк ей верно служит;
Там ступа с Бабою Ягой
Идёт, бредёт сама собой,
Там царь Кащей над златом чахнет;
Там русский дух… там Русью пахнет!
И там я был, и мёд я пил;
У моря видел дуб зелёный;
Под ним сидел, и кот учёный
Свои мне сказки говорил.

